In [1]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [2]:
dataset = load_dataset("chirunder/text_messages")

Generating train split:   0%|          | 0/11615290 [00:00<?, ? examples/s]

In [3]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [4]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [ ]:
# Cargar el tokenizer del modelo openai-community/gpt2 de Huggingface
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Embeddear las letras de las canciones
tokenized_text = df['texto'].apply(lambda x: tokenizer(x)["input_ids"])

tokenized_text = tokenized_text.apply(lambda x: x[:1000] + [0] * (1000 - len(x)) if len(x) < 1000 else x[:1000])

# Create a new DataFrame with 'track_name' and the tokenized lyrics
df_tokenized = pd.DataFrame({'vector': tokenized_text, 'name': df['texto']})

df_tokenized.head()

/usr/local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [ ]:
db = lancedb.connect("./.lancedb")
# Creamos una tabla en la base de datos
db.create_table("tabla", df_tokenized)
db["tabla"].head()

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [ ]:
(db["tabla"].search()
    .where("LENGTH(texto) > 100")
    .select(["name", "vector"])
    .limit(5)
    .to_pandas())

- Query en SQL equivalente: select texto from tabla where len(texto) >100 limit 5
- 
- Explicacion: Me basé mucho en la clase, solamente que como son textos quería ver si había oraciones que juntaran más de 100 porque me dio curiosidad 

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [ ]:
mensaje= "i think therefore i am i AM"
mensaje_embedded = tokenizer(mensaje)["input_ids"]

In [ ]:
n = 1000
def ajustar_vector(input, n):
    output = input[:n]
    
    # Si la lista es más corta que el tamaño objetivo, rellenar con 0.0
    while len(output) < n:
        output.append(0)
    
    return output
mensaje_embedded = ajustar_vector(mensaje_embedded, n)

In [ ]:
(db["tabla"].search(mensaje_embedded)
    .metric("cosine") 
    .where("name != 'i think therefore i am i AM'")
    .select(["name", "vector"])
    .limit(1)
    .to_pandas())